In [6]:
import torch
import torchvision.transforms as T
from PIL import Image
import os
import matplotlib.pyplot as plt
import numpy as np
import qai_hub as hub
from torchcam.methods import GradCAM
import segmentation_models_pytorch as smp
import torch.nn as nn

In [8]:
class UNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = smp.UnetPlusPlus(
            encoder_name='timm-mobilenetv3_small_100',
            encoder_weights='imagenet',
            in_channels=3,
            classes=1  # Outputting a single channel for depth map
        )
        
    def trainable_encoder(self, trainable=True):
        for p in self.model.encoder.parameters():
            p.requires_grad = trainable
        
    def forward(self, x):
        return self.model(x)
    
    def _num_params(self):
        return sum([p.numel() for p in self.model.parameters() if p.requires_grad])



In [11]:
import os

print(os.listdir('/kaggle/input/cvt1/pytorch/default/1'))


['nyu-v2-depth-mobilenet-unetplusplus.pt']


In [12]:
import torch
from collections import OrderedDict

# Define your UNet model
unet_model = UNet().to('cuda')

# Load the state dictionary from the checkpoint file
state_dict = torch.load('/kaggle/input/cvt1/pytorch/default/1/nyu-v2-depth-mobilenet-unetplusplus.pt', map_location='cuda')

# Remove "module." prefix if present
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    new_key = k.replace("module.", "")
    new_state_dict[new_key] = v

# Load the state dict into the model
unet_model.load_state_dict(new_state_dict)
unet_model.eval()


<ipython-input-12-7ac3c5624b2e>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('/kaggle/input/cvt1/pytorch/default/1/nyu-v2-depth-mobilenet-unetplu

UNet(
  (model): UnetPlusPlus(
    (encoder): MobileNetV3Encoder(
      (model): MobileNetV3Features(
        (conv_stem): Conv2dSame(3, 16, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (bn1): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (act1): Hardswish()
        (blocks): Sequential(
          (0): Sequential(
            (0): DepthwiseSeparableConv(
              (conv_dw): Conv2dSame(16, 16, kernel_size=(3, 3), stride=(2, 2), groups=16, bias=False)
              (bn1): BatchNormAct2d(
                16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
                (drop): Identity()
                (act): ReLU(inplace=True)
              )
              (aa): Identity()
              (se): SqueezeExcite(
                (conv_reduce): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
                (act1): ReLU(inplace=True)
                (conv_expand): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
       

In [13]:
! qai-hub configure --api_token f4840767cb173bbf00ee11945958efe528d14c63


qai-hub configuration saved to /root/.qai_hub/client.ini
==================== /root/.qai_hub/client.ini ====================
[api]
api_token = f4840767cb173bbf00ee11945958efe528d14c63
api_url = https://app.aihub.qualcomm.com
web_url = https://app.aihub.qualcomm.com
verbose = True




In [15]:
# ---------------------------
# Trace the model
# ---------------------------
example_input = torch.rand((1, 3, 480, 640))
traced_model = torch.jit.trace(unet_model, example_input)

# ---------------------------
# Compile with AI Hub
# ---------------------------
device = hub.Device("Samsung Galaxy S24 (Family)")

compile_job = hub.submit_compile_job(
    name='unetplusplus_depth_estimation',
    model=traced_model,
    device=device,
    input_specs={"image": (1, 3, 480, 640)}
)

print("⏳ Compiling model...")
compile_job.wait()

ModuleNotFoundError: No module named 'samsung_ai_hub'

In [5]:
# Download compiled artifacts
compile_job.download_results(artifacts_dir="./compile_job_results")
compiled_model = compile_job.get_target_model()

job_j5qeqy4m5_optimized_tflite_mnjlk6dxq.tflite: 100%|██████████| 185M/185M [00:12<00:00, 15.5MB/s] 


Downloaded model to e:\College\S6\CV\indoor-monocular-depth-estimation\unet-plus-plus-depth\compile_job_results\job_j5qeqy4m5_optimized_tflite_mnjlk6dxq.tflite


In [6]:
# ---------------------------
# Profile the compiled model
# ---------------------------
profile_job = hub.submit_profile_job(
    name='unetplusplus_depth_profile',
    model=compiled_model,
    device=device
)

print("⏳ Profiling on device...")
profile_job.wait()
profile_job.download_results(artifacts_dir="./profile_job_results")

Scheduled profile job (j56e47o7g) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/j56e47o7g/

⏳ Profiling on device...
Waiting for profile job (j56e47o7g) completion. Type Ctrl+C to stop waiting at any time.
    ✅ SUCCESS                          
Saved profile results to e:\College\S6\CV\indoor-monocular-depth-estimation\unet-plus-plus-depth\profile_job_results\unetplusplus_depth_profile_j56e47o7g_results.json


ProfileJobResult
----------------
status                        : JobStatus
---------
code    : SUCCESS
message : 

url                           : https://app.aihub.qualcomm.com/jobs/j56e47o7g/
artifacts_dir                 : e:\College\S6\CV\indoor-monocular-depth-estimation\unet-plus-plus-depth\profile_job_results
Estimated Inference Time (ms) : 60.711
Load Time (ms)                : 10844.763
Peak Memory (MB)              : 743.68359375
Compute Units (layers)        : NPU: 121

In [12]:
profile_result = profile_job.download_profile()
print("\n✅ Profile Job Completed!")
print(f"📄 Name: {profile_job.name}")
print(f"📱 Target Device: {profile_job.device.name}")


✅ Profile Job Completed!
📄 Name: unetplusplus_depth_profile
📱 Target Device: Samsung Galaxy S24 (Family)


In [ ]:
# # ---------------------------
# # Run Inference on Test Images
# # ---------------------------
# transform = T.Compose([
#     T.Resize((480, 640)),
#     T.ToTensor()
# ])

# def load_image(path):
#     img = Image.open(path).convert('RGB')
#     return transform(img).unsqueeze(0)

# def save_depth_map(depth, output_path):
#     depth_np = depth.squeeze().detach().cpu().numpy()
#     plt.imsave(output_path, depth_np, cmap='inferno')

# # Load test images
# test_dir = '../test_data/indoor'
# test_images = [os.path.join(test_dir, f) for f in os.listdir(test_dir) if f.endswith(('.jpg', '.png'))]
# os.makedirs("depth_predictions", exist_ok=True)

# for i, img_path in enumerate(test_images):
#     img_tensor = load_image(img_path)
#     with torch.no_grad():
#         pred = unet_model(img_tensor)
#         save_path = f"depth_predictions/depth_{i+1}.png"
#         save_depth_map(pred, save_path)

# print(f"\n🖼️ Saved {len(test_images)} predicted depth maps to 'depth_predictions/' folder.")


🖼️ Saved 100 predicted depth maps to 'depth_predictions/' folder.
